In [1]:
#Import statements
import json
import requests
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import os
import csv
import time

In [2]:
# The stock Symbols that we will iterate through
# Apple = AAPL
# Amazon = AMZN
# Google = GOOGL
# Microsoft = MSFT
# Dell = DELL
# IBM = IBM
# Intel = INTC
# HP = HPQ
# Facebook = FB
# Cisco Systems = CSCO
# Oracle = ORCL
# HP Enterprise = HPE
# Micron Tech = MU
# DXC Tech = DXC
# Thermo Fisher Scientific = TMO

# Qualcomm = QCOM
# Jabil = JBL
# Broadcom = AVGO
# Western Digital = WDC
# Applied Materials = AMAT


# Tesla = TSLA
# Alibaba = BABA
# Nvidia = NVDA
# Walmart = WMT
# Walt Disney = DIS
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [ ]:
#Gathers the json file and all needed data values.
def collect_Twits(res):
    try:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})


    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    #returns the dataframe in correct format
    return df

In [1]:
#Gathers the json file and all needed data values.
def collect_New_Twits(res):
    try:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})


    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    #Newest posts will be on top
    df.sort_values('created', ascending=False)
    
    #returns the dataframe in correct format
    return df

In [ ]:
while True:
    for symbol in stockSymbol:
    
        #selects the file to add to
        file = '{}_newTwits.csv'.format(symbol)
    
        historicalTwits = pd.read_csv(file)
        newestID = historicalTwits['id'].iloc[0]
    
        url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
        response = requests.get(url, params = {'since' :  newestID}).json()
    
        tempNewTwitsDf = collect_New_Twits(response)

        newHistoricalTwits = tempTwitsDf.append(historicalTwits)
    
        newHistoricalTwits.to_csv(r'{}_newTwits.csv'.format(symbol), index=False)
        time.sleep(30)
        newHistoricalTwits.to_csv(r'{}_newTwitsCopy.csv'.format(symbol), index=False)
        time.sleep(30)